## Импортирование используемых библиотек

In [2]:
import pandas as pd
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

## Блок со вспомогательными функциями

In [30]:
def prepare_text(text):
    doc = Doc(text)
    doc.segment(segmenter)      #разбиваем текст на токены и предложения
    doc.tag_morph(morph_tagger) #для каждого токена получаем морфологические метки
    #лемматизируем каждый токен
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return doc

#проверяем, есть ли лемма в списке приветствий
def detect_greeting(token):
    if token.lemma in greeting_list:
        return True
    return False

#проверяем, есть ли лемма в списке представлений
def detect_manager_name(token):
    maybe_name = False
    if token.lemma in representation_list:
        maybe_name = True
    return maybe_name

#извлекаем название компаний
def detect_company(token, company_name, company_flag):
    #первое слово в тексте после слова "компания" обязательно добавляем в буфер
    if company_flag == 2:
        company_name += token.text
        company_flag = 1
    elif company_flag:
        #добавление слов в буфер заканчиваем, если встретили местоимение или императивный глагол
        if (token.pos in ['PRON', 'ADJ']) or ((token.pos == 'VERB') and (token.feats['Aspect'] == 'Imp')):
            company_flag = 0
            return company_name, company_flag
        company_name += ' '
        company_name += token.text
    elif token.lemma == 'компания':
        company_flag = 2
    return company_name, company_flag

#проверяем, попрощался ли менеджер
def check_manager_goodbye(df, i):
    manager_replica_number_invert = 3   #счетчик количества реплик менеджера -
                                        #нужен, чтобы проверять прощание только
                                        #в последних трёх репликах
    manager_said_bye = False

    while manager_replica_number_invert != 0:
        i -= 1
        line = df.iloc[i]
        if line['role'] == 'manager':
            manager_replica_number_invert -= 1
            #проверяем наличие фраз из списка с прощаниями в исследуемой реплике
            for goodbye_phrase in goodbye_list:
                if goodbye_phrase in line['text'].lower():
                    df.at[i, 'isGoodbye'] = True
                    manager_said_bye = True
                    break
    return manager_said_bye

## Объявление переменных

In [4]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
embedding = NewsEmbedding()
morph_tagger = NewsMorphTagger(embedding)

In [76]:
greeting_list = ['добрый', 'здравствовать', 'приветствовать']
goodbye_list = ['до свидания', 'всего хорошего', 'всего доброго',
                'удачи', 'счастливо', 'до завтра', 'до понедельника',
                'до вторника', 'до среды', 'до четверга', 'до пятницы',
                'до субботы', 'до воскресенья', 'хорошего дня',
                'хорошего вечера']
representation_list = ['я', 'звать', 'имя', 'это']

## Считывание данных из файла

In [77]:
df = pd.read_csv('test_data.csv')
df[['isGreeting', 'isGoodbye', 'isIntroducing']] = False
df[['nameManager', 'nameCompany']] = None

## Парсинг данных

In [81]:
dialog_id = 0                   #номер диалога
manager_replica_number = 0      #счетчик количества реплик менеджера - нужен, чтобы 
                                #проверять приветствие только в первых трёх репликах
manager_polite_list = [[0, 0]]  #список из списков - первый элемент вложенного списка 
                                #отвечает за то, поздоровался ли менеджер, 
                                #второй - попрощался ли

for i in range(len(df)):
    line = df.iloc[i]

    #если предыдущий диалог закончился, то меняем значение dialog_id,
    #и проверяем, попрощался ли менеджер
    if dialog_id != line['dlg_id']:
        dialog_id = line['dlg_id']
        if check_manager_goodbye(df, i):
            manager_polite_list[dialog_id - 1][1] = 1
        manager_replica_number = 0
        manager_polite_list.append([0, 0])

    if manager_replica_number < 3:
        #проверяем только реплики менеджера
        if line['role'] == 'manager':
            manager_replica_number += 1
            doc = prepare_text(line['text'])

            is_greeting = False
            manager_name = ''       #буфер для имени менеджера
            manager_flag = False    #флаг - активирует добавление слов в буфер менеджера
            company_name = ''       #буфер для названия компании
            company_flag = 0        #флаг - активирует добавление слов в буфер компании

            for j in range(len(doc.tokens)):
                token = doc.tokens[j]
                #определяем, является ли токен приветствием
                if not is_greeting:
                    is_greeting = detect_greeting(token)
                #определяем имя менеджера
                if not manager_name:
                    #проверяем, был ли предыдущий токен был из списка представлений
                    if manager_flag:
                        manager_flag = False
                        #добавляем текущий токен как имя менеджера, если это 
                        #одушевлённое существительное или если поблизости есть слово "звать"
                        if (token.pos == 'NOUN') and (token.feats['Animacy'] == 'Anim'):
                            manager_name = token.text
                        elif ((doc.tokens[j-1].lemma == 'я') and 
                              ((doc.tokens[j-2].lemma == 'звать') or 
                               (doc.tokens[j+1].lemma == 'звать'))):
                            manager_name = token.text
                    manager_flag = detect_manager_name(token)
                #определяем имя компании
                company_name, company_flag = detect_company(token, company_name, company_flag)

            if is_greeting:
                df.at[i, 'isGreeting'] = True   #добавляем запись в датафрейм,
                                                #если менеджер здоровался
                manager_polite_list[dialog_id][0] = 1

            if manager_name:
                df.at[i, 'nameManager'] = manager_name  #добавляем запись в датафрейм,
                                                        #если нашли имя менеджера
                df.at[i, 'isIntroducing'] = True    #добавляем запись в датафрейм,
                                                    #если менеджер представился

            if company_name:
                df.at[i, 'nameCompany'] = company_name  #добавляем запись в датафрейм,
                                                        #если нашли название компании

#так как для последнего диалога не проверили прощание менеджера - делаем это вне цикла
if check_manager_goodbye(df, i + 1):
    manager_polite_list[dialog_id][1] = 1

## Вывод результата для каждого диалога

In [ ]:
for i in range(dialog_id + 1):
    print('Диалог', i)
    print('Реплики, где менеджер поздоровался:', df[(df.dlg_id == i) & (df.isGreeting == True)].text.values)
    print('Реплики, где менеджер представился:', df[(df.dlg_id == i) & (df.isIntroducing == True)].text.values)
    print('Имя менеджера:', df[(df.dlg_id == i) & (df.nameManager)].nameManager.values)
    print('Название компании:', df[(df.dlg_id == i) & (df.nameCompany)].nameCompany.values)
    print('Реплики, где менеджер попрощался:', df[(df.dlg_id == i) & (df.isGoodbye == True)].text.values)
    print('Менеджер поздоровался и попрощался:', manager_polite_list[i] == [1, 1])
    print()

### Вывод всего датафрейма

In [ ]:
df